In [0]:
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Usar Spark para cargar la tabla en un DataFrame de Spark
df_spark = spark.table("default.saberpro_csv")

# Convertir el DataFrame de Spark a un DataFrame de Pandas
df_SaberPro = df_spark.toPandas()

# Mostrar los primeros registros del DataFrame resultante
print(df_SaberPro.head())

columnas_interes = ['FAMI_ESTRATOVIVIENDA','FAMI_EDUCACIONMADRE','FAMI_EDUCACIONPADRE','FAMI_TIENEAUTOMOVIL','FAMI_TIENECOMPUTADOR','FAMI_TIENEINTERNET','FAMI_TIENELAVADORA']

#label_encoder = LabelEncoder()
#for columna in columnas_interes:
#    df_Saber11[columna] = label_encoder.fit_transform(df_Saber11[columna])

label_encoder = LabelEncoder()
for columna in columnas_interes:
    df_SaberPro[columna] = label_encoder.fit_transform(df_SaberPro[columna])

columns_to_sum = [
    'MOD_COMPETEN_CIUDADA_PUNT',
    'MOD_LECTURA_CRITICA_PUNT',
    'MOD_RAZONA_CUANTITAT_PUNT',
    'MOD_COMUNI_ESCRITA_PUNT',
    'MOD_INGLES_PUNT'
]

# Convertir las columnas a numéricas
for column in columns_to_sum:
    df_SaberPro[column] = pd.to_numeric(df_SaberPro[column], errors='coerce')

# Crear la columna PUNT_GLOBAL como la suma de las columnas
df_SaberPro['PUNT_GLOBAL'] = df_SaberPro[columns_to_sum].sum(axis=1)

for column in columns_to_sum:
    df_SaberPro[column] = df_SaberPro[column].astype(str)

# Preparar los datos
y = df_SaberPro['PUNT_GLOBAL']  # Variable objetivo
X = df_SaberPro.drop(columns=['PUNT_GLOBAL'])  # Resto de las columnas

# Seleccionar solo las columnas numéricas
numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns
X_numerical = X[numerical_columns]

# Estandarizar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_numerical)

# Reducir las dimensiones con PCA
pca = PCA(n_components=6)  
X_pca = pca.fit_transform(X_scaled)

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# registre el experimento
experiment = mlflow.set_experiment("/Users/s.barrera@uniandes.edu.co/Saber_pro_MFLOW")

# Aquí se ejecuta MLflow sin especificar un nombre o id del experimento. MLflow los crea un experimento para este cuaderno por defecto y guarda las características del experimento y las métricas definidas. 
# Para ver el resultado de las corridas haga click en Experimentos en el menú izquierdo. 
with mlflow.start_run(experiment_id=experiment.experiment_id):
    # defina los parámetros del modelo
    n_estimators = 50
    max_depth = 2
    max_features = 2
    # Cree el modelo con los parámetros definidos y entrénelo
    rf = RandomForestRegressor(n_estimators = n_estimators, max_depth = max_depth, max_features = max_features)
    rf.fit(X_train, y_train)
    # Realice predicciones de prueba
    predictions = rf.predict(X_test)
  
    # Registre los parámetros
    mlflow.log_param("num_trees", n_estimators)
    mlflow.log_param("maxdepth", max_depth)
    mlflow.log_param("max_feat", max_features)
  
    
    # Registrar el modelo con ejemplo de entrada
    mlflow.sklearn.log_model(rf, "random-forest-model")
    # Cree y registre la métrica de interés
    mse = mean_squared_error(y_test, predictions)
    mlflow.log_metric("mse", mse)
    print(mse)


  ESTU_COD_RESIDE_DEPTO ESTU_COD_RESIDE_MCPIO  ...   AÑO EDAD_EXAMEN
0                  11.0               11001.0  ...  2018        21.0
1                  76.0               76736.0  ...  2018        23.0
2                  73.0               73001.0  ...  2018        24.0
3                  68.0               68081.0  ...  2018        21.0
4                  11.0               11001.0  ...  2018        31.0

[5 rows x 42 columns]


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

🏃 View run monumental-panda-789 at: https://community.cloud.databricks.com/ml/experiments/2818553716847128/runs/e9a3bf4dd9464799a95ea19ffd5115e1
🧪 View experiment at: https://community.cloud.databricks.com/ml/experiments/2818553716847128


2024/11/26 07:02:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

14368.007438930033
🏃 View run stylish-steed-586 at: https://community.cloud.databricks.com/ml/experiments/2818553716847128/runs/7183ee3a8f444f0db81a5d705eae160b
🧪 View experiment at: https://community.cloud.databricks.com/ml/experiments/2818553716847128
